<a href="https://colab.research.google.com/github/saheedniyi02/naijaweb/blob/main/fineweb_clean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
files=os.listdir("/content/drive/MyDrive/nairaland_webtext/cleaned_403")

In [ ]:
all_dfs=[]
for file in files:
    df=pd.read_csv(f"/content/drive/MyDrive/nairaland_webtext/cleaned_403/{file}")
    print(file)
    all_dfs.append(df)

data_33.csv
data_99.csv
data_66.csv
data_116.csv


In [ ]:
files=os.listdir("/content/drive/MyDrive/nairaland_webtext/cleaned_dataset")

In [ ]:
for file in files:
    df=pd.read_csv(f"/content/drive/MyDrive/nairaland_webtext/cleaned_dataset/{file}")
    print(file)
    all_dfs.append(df)

data_133.csv
data_166.csv
data_199.csv
data_233.csv
data_266.csv
data_300.csv
data_933.csv
data_945.csv
data_833.csv
data_733.csv
data_866.csv
data_766.csv
data_900.csv
data_800.csv
data_466.csv
data_533.csv
data_433.csv
data_566.csv
data_333.csv
data_366.csv
data_633.csv
data_700.csv
data_666.csv
data_400.csv
data_500.csv
data_600.csv
data_100.csv
data_66.csv
data_33.csv
data_invalid1.csv


In [ ]:
files=os.listdir("/content/drive/MyDrive/openweb_naij/cleaned_data")

In [ ]:
for file in files:
    df=pd.read_csv(f"/content/drive/MyDrive/openweb_naij/cleaned_data/{file}")
    print(file)
    all_dfs.append(df)

data_500.csv
data_1000.csv
data_1500.csv
data_2000.csv
data_2400.csv
data_403.csv


In [ ]:
data=pd.concat(all_dfs)

In [ ]:
data.shape

(597091, 4)

In [ ]:
data=data[~pd.isna(data["text"])]
data.drop_duplicates("url", inplace=True)

In [ ]:
data.drop_duplicates("text", inplace=True)

In [ ]:
data.shape

(350095, 4)

In [ ]:
pip install datatrove[io,processing] tldextract newspaper3k trafilatura

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 35.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 6.0 MB/s eta 0:00:

### Datatrove

In [ ]:
from datatrove.data import Document
from datatrove.pipeline.extractors import Trafilatura
from datatrove.pipeline.filters import (
    C4QualityFilter,
    FineWebQualityFilter,
    GopherQualityFilter,
    GopherRepetitionFilter,
    LanguageFilter,
    URLFilter,
)
from datatrove.pipeline.readers import WarcReader
from datatrove.pipeline.writers.jsonl import JsonlWriter
from datatrove.pipeline.writers.parquet import ParquetWriter
from datatrove.pipeline.tokens import TokensCounter

In [ ]:
#from importlib.metadata import metadata
def get_data():
  docs=[]
  for i,text in enumerate(data["text"]):
    url=data["url"].iloc[i]
    #text=result["text"]
    if (text is not None) and (len(text)>0):
         docs.append(Document(text=text, id=i,metadata={"url":url}))
  return docs

In [ ]:
MAIN_OUTPUT_PATH="/content/drive/MyDrive/nairaland_webtext/processed_final"
if not os.path.exists(MAIN_OUTPUT_PATH):
    os.makedirs(MAIN_OUTPUT_PATH)

if not os.path.exists(f"{MAIN_OUTPUT_PATH}/removed/url"):
    os.makedirs(f"{MAIN_OUTPUT_PATH}/removed/url")

if not os.path.exists(f"{MAIN_OUTPUT_PATH}/removed/repetitive"):
    os.makedirs(f"{MAIN_OUTPUT_PATH}/removed/repetitive")

if not os.path.exists(f"{MAIN_OUTPUT_PATH}/removed/quality"):
    os.makedirs(f"{MAIN_OUTPUT_PATH}/removed/quality")

if not os.path.exists(f"{MAIN_OUTPUT_PATH}/removed/c4"):
    os.makedirs(f"{MAIN_OUTPUT_PATH}/removed/c4")

if not os.path.exists(f"{MAIN_OUTPUT_PATH}/removed/fineweb_qual"):
    os.makedirs(f"{MAIN_OUTPUT_PATH}/removed/fineweb_qual")

In [ ]:
pipeline=[
        get_data(),
        URLFilter(exclusion_writer=JsonlWriter(f"{MAIN_OUTPUT_PATH}/removed/url")),
        GopherRepetitionFilter(
            exclusion_writer=JsonlWriter(f"{MAIN_OUTPUT_PATH}/removed/repetitive")
            ),
        GopherQualityFilter(
            exclusion_writer=JsonlWriter(f"{MAIN_OUTPUT_PATH}/removed/quality")
            ),
         C4QualityFilter(
            filter_no_terminal_punct=False,
            exclusion_writer=JsonlWriter(f"{MAIN_OUTPUT_PATH}/removed/c4"),
        ),
        FineWebQualityFilter(
            exclusion_writer=JsonlWriter(f"{MAIN_OUTPUT_PATH}/removed/fineweb_qual")
        ),
        TokensCounter(),
        JsonlWriter(f"{MAIN_OUTPUT_PATH}"),
        #Jr(f"{MAIN_OUTPUT_PATH}"),
    ]

In [ ]:
from datatrove.executor import LocalPipelineExecutor
executor = LocalPipelineExecutor(
    pipeline=pipeline,
    logging_dir="main_logs",
    #tasks=10,
    #workers=5,
)

In [ ]:
executor.run()

2024-10-03 09:58:43.866 | INFO     | datatrove.utils.logging:add_task_logger:58 - Launching pipeline for rank=0
2024-10-03 09:58:43.874 | INFO     | datatrove.utils.logging:log_pipeline:90 - 
--- 🛠️ PIPELINE 🛠
Iterable
🔻 - FILTER: 😈 Url-filter
🔻 - FILTER: 👯 Gopher Repetition
🔻 - FILTER: 🥇 Gopher Quality
🔻 - FILTER: ⛰ C4 Quality
🔻 - FILTER: 🍷 FineWeb Quality
🔢 - TOKENIZER: 📊 Counter
💽 - WRITER: 🐿 Jsonl
2024-10-03 09:58:43.893 | INFO     | datatrove.pipeline.filters.url_filter:do_extract:87 - 💥 Extracting url filter blacklists...
2024-10-03 09:58:45.291 | INFO     | datatrove.pipeline.filters.url_filter:do_extract:90 - 💥 Extracted url filter blacklists.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error pers

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2024-10-03 13:08:27.861 | SUCCESS  | datatrove.executor.base:_run_for_rank:98 - Processing done for rank=0
2024-10-03 13:08:27.950 | INFO     | datatrove.executor.base:_run_for_rank:104 - 

📉📉📉 Stats: Task 0 📉📉📉

Total Runtime: 3 hours, 6 minutes and 51 seconds

🔻 - FILTER: 😈 Url-filter
    Runtime: (0.73%) 1 minute and 22 seconds [0.23 milliseconds±14.08 milliseconds/doc]
    Stats: {total: 350095, forwarded: 349281, doc_len: 1398798842 [min=1, max=5852385, 4004.80±17662/doc], dropped: 814, dropped_domain: 344, dropped_hard_blacklisted: 225, dropped_soft_blacklisted: 124, dropped_blacklisted_subword: 117, dropped_subdomain: 4}
🔻 - FILTER: 👯 Gopher Repetition
    Runtime: (41.96%) 1 hour, 18 minutes and 24 seconds [13.47 milliseconds±59.94 milliseconds/doc]
    Stats: {total: 349281, forwarded: 322684, doc_len: 1268143258 [min=1, max=3351750, 3929.98±12350/doc], dropped: 26597, dropped_top_2_gram: 2549, dropped_duplicated_5_n_grams: 5206, dropped_top_4_gram: 2701, dropped_dup_line_frac



📉📉📉 Stats 📉📉📉

Total Runtime: 3 hours, 6 minutes and 51 seconds

🔻 - FILTER: 😈 Url-filter
    Runtime: (0.73%) 1 minute and 22 seconds [0.23 milliseconds±14.08 milliseconds/doc]
    Stats: {total: 350095, forwarded: 349281, doc_len: 1398798842 [min=1, max=5852385, 4004.80±17662/doc], dropped: 814, dropped_domain: 344, dropped_hard_blacklisted: 225, dropped_soft_blacklisted: 124, dropped_blacklisted_subword: 117, dropped_subdomain: 4}
🔻 - FILTER: 👯 Gopher Repetition
    Runtime: (41.96%) 1 hour, 18 minutes and 24 seconds [13.47 milliseconds±59.94 milliseconds/doc]
    Stats: {total: 349281, forwarded: 322684, doc_len: 1268143258 [min=1, max=3351750, 3929.98±12350/doc], dropped: 26597, dropped_top_2_gram: 2549, dropped_duplicated_5_n_grams: 5206, dropped_top_4_gram: 2701, dropped_dup_line_frac: 9128, dropped_dup_line_char_frac: 2247, dropped_duplicated_6_n_grams: 416, dropped_duplicated_10_n_grams: 638, dropped_top_3_gram: 2390, dropped_duplicated_7_n_grams: 380, dropped_duplicated_9_n